# **Проект по оптимизации простейших операций над комплексными числам на Python с использованием библиотек Cython и Numpy**

Оформили проект студенты группы ПМ-2001: Киселёв Григорий и Давлетшин Азат

---

### **Установка нужных библиотек**



In [ ]:
!pip install Cython
!pip install ipdb
!pip install git+https://github.com/RJT1990/pyflux
!pip install ipython-autotime

%load_ext cython

import numpy as np
import Cython
from time import time

In [ ]:
%load_ext autotime

---

### **Создание класса и описание методов для работы с комплексными числами с Pure Python**




In [7]:
from math import *

class PPComplexNum:

  def __init__(self, a, b):
    self.re = a
    self.im = b
    self.norm = sqrt(self.re * self.re + self.im * self.im)
    try: 
      div = self.im / self.re
    except ArithmeticError:
      self.angle = pi / 2 if self.im > 0 else 3 * pi / 2 
    else:
      self.angle = atan(div) + pi if self.re < 0 else atan(div)

  def alg_print(self):
    return f"{self.re} + i * {self.im}"

  def trig_print(self):
    return f"{self.norm} * (cos({self.angle}) + i * sin({self.angle}))"

  def exp_print(self):
    return f"{self.norm} * e ^ (i * {self.angle})"
  
  def __add__(self, OtherComplex):
    return PPComplexNum(self.re + OtherComplex.re, self.im + OtherComplex.im)
  
  def __sub__(self, OtherComplex):
    return PPComplexNum(self.re - OtherComplex.re, self.im - OtherComplex.im)
  
  def __mul__(self, OtherComplex):
    return PPComplexNum(self.re * OtherComplex.re - OtherComplex.im * self.im, 
                        self.re * OtherComplex.im + self.im * OtherComplex.re)
  
  def __truediv__(self, OtherComplex):
    if (OtherComplex.im == 0 and OtherComplex.re == 0):
      print("Деление на ноль")
      return
    return PPComplexNum((self.re * OtherComplex.re + self.im * OtherComplex.im) 
    / (OtherComplex.re ** 2 + OtherComplex.im ** 2), 
    (OtherComplex.re * self.im - self.re * OtherComplex.im) / 
    (OtherComplex.re ** 2 + OtherComplex.im ** 2))
  
  def cln(self):
    return f"ln({self.norm}) + i * ({self.angle} + 2pi * n, n - целое)"
  
  def pow(self, OtherComplex):
    return f"e^(({OtherComplex.re} + i * {OtherComplex.im}) * (" + self.cln() + ")"


pn1 = PPComplexNum(2.12, 5.11)
pn2 = PPComplexNum(3.14, -4)
print("Время инициализации для Pure Python:")

Время инициализации для Pure Python:
time: 7.31 ms (started: 2022-05-31 20:07:23 +00:00)


---

### **Создание класса и описание методов для работы с комплексными числами с Cython**


In [8]:
%%cython
import cython
from libc.math cimport sqrt, M_PI, atan

class CPComplexNum:
  
  def __init__(self, a, b):
    self.re = a
    self.im = b
    cdef float re = self.re
    cdef float im = self.im
    cdef float angle
    cdef float div
    cdef float norm = sqrt(re * re + im * im)
    self.norm = norm
    try: 
      div = im / re
    except ArithmeticError:
      angle = M_PI / 2 if re > 0 else 3 * M_PI / 2 
    else:
      angle = atan(div) + M_PI if re < 0 else atan(div)
    self.angle = angle

  def alg_print(self):
    return f"{self.re} + i * {self.im}"

  def trig_print(self):
    return f"{self.norm} * (cos({self.angle}) + i * sin({self.angle}))"

  def exp_print(self):
    return f"{self.norm} * e ^ (i * {self.angle})"

  def __add__(self, OtherComplex):
    cdef float a1 = self.re
    cdef float b1 = self.im
    cdef float a2 = OtherComplex.re
    cdef float b2 = OtherComplex.im
    return CPComplexNum(a1 + a2, b1 + b2)
  
  def __sub__(self, OtherComplex):
    cdef float a1 = self.re
    cdef float b1 = self.im
    cdef float a2 = OtherComplex.re
    cdef float b2 = OtherComplex.im
    return CPComplexNum(a1 - a2, b1 - b2)
  
  def __mul__(self, OtherComplex):
    cdef float a1 = self.re
    cdef float b1 = self.im
    cdef float a2 = OtherComplex.re
    cdef float b2 = OtherComplex.im
    return CPComplexNum(a1 * a2 - b2 * b1, a1 * b2 + b1 * a2)
  
  def __truediv__(self, OtherComplex):
    cdef float a1 = self.re
    cdef float b1 = self.im
    cdef float a2 = OtherComplex.re
    cdef float b2 = OtherComplex.im
    if (a2 == 0 and b2 == 0):
      print("Деление на ноль")
      return
    return CPComplexNum((a1 * a2 + b1 * b2) 
    / (a2 ** 2 + b2 ** 2), (a2 * b1 - a1 * b2) / (a2 ** 2 + b2 ** 2))
  
  def cln(self):
    return f"ln({self.norm}) + i * ({self.angle} + 2pi * n, n - целое)"
  
  def pow(self, OtherComplex):
    return f"e^(({OtherComplex.re} + i * {OtherComplex.im}) * (" + self.cln() + ")"

cn1 = CPComplexNum(2.12, 5.11)
cn2 = CPComplexNum(3.14, -4)
print("Время инициализации для Cython:")

Время инициализации для Cython:
time: 1.67 s (started: 2022-05-31 20:07:23 +00:00)


---

### **Создание класса и описание методов для работы с комплексными числами с NumPy**



In [9]:
class NumPyComplexNum:
  def __init__ (self, a, b):
    self.comp = a + b*1j
    self.norm = np.absolute(self.comp)
    self.angle = np.angle(self.comp)
  def __str__(self):
    return f"{self.comp}"
  def __add__(self, OtherComplex):
    return np.array([self.comp + OtherComplex.comp])
  def __sub__(self, OtherComplex):
    return self.comp - OtherComplex.comp
  def __mul__(self, OtherComplex):
    return np.multiply(self.comp, OtherComplex.comp)
  def __truediv__(self, OtherComplex):
    return np.divide(self.comp, OtherComplex.comp)
  def cln(self):
    return np.log(self.comp)
  def pow(self, OtherComplex):
    return np.power(self.comp, OtherComplex.comp)

nn1 = NumPyComplexNum(2.12, 5.11)
nn2 = NumPyComplexNum(3.14, -4)
print("Время инициализации: ")

Время инициализации: 
time: 3.22 ms (started: 2022-05-31 20:07:24 +00:00)


---

## **Сравнение времени выполнения задач**




---

***Вывод комлпексного числа***

In [10]:
print("Алгебраическая форма записи (PyrePython): ", pn1.alg_print())

Алгебраическая форма записи (PyrePython):  2.12 + i * 5.11
time: 606 µs (started: 2022-05-31 20:07:24 +00:00)


In [11]:
print("Алгебраическая форма записи (Cython): ", cn1.alg_print())

Алгебраическая форма записи (Cython):  2.12 + i * 5.11
time: 1.35 ms (started: 2022-05-31 20:07:24 +00:00)


In [12]:
print("Алгебраическая форма записи (NumPy) ", nn1)
print("Алгебраическая форма записи (NumPy) ", nn2)

Алгебраическая форма записи (NumPy)  (2.12+5.11j)
Алгебраическая форма записи (NumPy)  (3.14-4j)
time: 697 µs (started: 2022-05-31 20:07:24 +00:00)


In [13]:
print("Тригонометрическая форма записи (PyrePython): ", pn1.trig_print())

Тригонометрическая форма записи (PyrePython):  5.532314163168972 * (cos(1.1775346830993867) + i * sin(1.1775346830993867))
time: 694 µs (started: 2022-05-31 20:07:24 +00:00)


In [14]:
print("Тригонометрическая форма записи (Cython): ", cn1.trig_print())

Тригонометрическая форма записи (Cython):  5.532314300537109 * (cos(1.1775346994400024) + i * sin(1.1775346994400024))
time: 727 µs (started: 2022-05-31 20:07:24 +00:00)


In [15]:
print("Экспоненциальная форма записи (PyrePython): ", pn1.exp_print())

Экспоненциальная форма записи (PyrePython):  5.532314163168972 * e ^ (i * 1.1775346830993867)
time: 787 µs (started: 2022-05-31 20:07:24 +00:00)


In [16]:
print("Экспоненциальная форма записи (Cython): ", cn1.exp_print())

Экспоненциальная форма записи (Cython):  5.532314300537109 * e ^ (i * 1.1775346994400024)
time: 1.19 ms (started: 2022-05-31 20:07:24 +00:00)


---

***Сумма двух комплексных чисел:***



In [17]:
print("PurePython: ", (pn1 + pn2).alg_print())

PurePython:  5.26 + i * 1.1100000000000003
time: 783 µs (started: 2022-05-31 20:07:24 +00:00)


In [18]:
print("Cython: ", (cn1 + cn2).alg_print())

Cython:  5.260000228881836 + i * 1.1100001335144043
time: 922 µs (started: 2022-05-31 20:07:24 +00:00)


In [19]:
print("NumPy: ", nn1 + nn2)

NumPy:  [5.26+1.11j]
time: 2.04 ms (started: 2022-05-31 20:07:24 +00:00)


---
***Разность двух комплексных чисел***

In [44]:
print("PurePython: ", (pn1 - pn2).alg_print())

PurePython:  -1.02 + i * 9.11
time: 718 µs (started: 2022-05-31 20:08:44 +00:00)


In [45]:
print("Cython: ", (cn1 - cn2).alg_print())

Cython:  -1.0200002193450928 + i * 9.110000610351562
time: 633 µs (started: 2022-05-31 20:08:45 +00:00)


In [40]:
print("NumPy: ", nn1 - nn2)

NumPy:  (-1.02+9.11j)
time: 1.05 ms (started: 2022-05-31 20:08:37 +00:00)


---

***Произведение двух комплексных чисел***

In [37]:
print("PurePython: ", (pn1 * pn2).alg_print())

PurePython:  27.0968 + i * 7.5654
time: 981 µs (started: 2022-05-31 20:08:27 +00:00)


In [38]:
print("Cython: ", (cn1 * cn2).alg_print())

Cython:  27.096799850463867 + i * 7.565401077270508
time: 725 µs (started: 2022-05-31 20:08:29 +00:00)


In [25]:
print("NumPy: ", nn1 * nn2)

NumPy:  (27.0968+7.5654j)
time: 768 µs (started: 2022-05-31 20:07:24 +00:00)


---

***Деление двух комплексных чисел***

In [26]:
print("PurePython: ", (pn1 / pn2).alg_print())

PurePython:  -0.5330012838558988 + i * 0.9484060078268806
time: 1.29 ms (started: 2022-05-31 20:07:24 +00:00)


In [27]:
print("Cython: ", (cn1 / cn2).alg_print())

Cython:  -0.5330013036727905 + i * 0.9484060406684875
time: 911 µs (started: 2022-05-31 20:07:24 +00:00)


In [46]:
print("PurePython: ", (pn1 / PPComplexNum(0, 0)))

Деление на ноль
PurePython:  None
time: 1.82 ms (started: 2022-05-31 20:09:04 +00:00)


In [29]:
print("Cython: ", (cn1 / CPComplexNum(0, 0)))

Деление на ноль
Cython:  None
time: 935 µs (started: 2022-05-31 20:07:24 +00:00)


In [30]:
print("NumPy: ", nn1 / nn2)

NumPy:  (-0.5330012838558987+0.9484060078268806j)
time: 841 µs (started: 2022-05-31 20:07:24 +00:00)


---

***Комплексный логарифм***

In [50]:
print("PurePython: ", pn1.cln())

PurePython:  ln(5.532314163168972) + i * (1.1775346830993867 + 2pi * n, n - целое)
time: 1.46 ms (started: 2022-05-31 20:09:17 +00:00)


In [48]:
print("Cython: ", cn1.cln())

Cython:  ln(5.532314300537109) + i * (1.1775346994400024 + 2pi * n, n - целое)
time: 846 µs (started: 2022-05-31 20:09:13 +00:00)


In [52]:
print("NumPy: ", nn1.cln())

NumPy:  (1.710606202350708+1.1775346830993867j)
time: 976 µs (started: 2022-05-31 20:09:21 +00:00)


---

***Вовзедение комплексного числа в комплексную степень***

In [53]:
print("PurePython: ", pn1.pow(pn2))

PurePython:  e^((3.14 + i * -4) * (ln(5.532314163168972) + i * (1.1775346830993867 + 2pi * n, n - целое))
time: 1.28 ms (started: 2022-05-31 20:09:28 +00:00)


In [62]:
print("Cython: ", cn1.pow(cn2))

Cython:  e^((3.14 + i * -4) * (ln(5.532314300537109) + i * (1.1775346994400024 + 2pi * n, n - целое))
time: 782 µs (started: 2022-05-31 20:09:43 +00:00)


In [58]:
print("NumPy: ", nn1.pow(nn2))

NumPy:  (-23895.286917230605+80.60510337504363j)
time: 858 µs (started: 2022-05-31 20:09:36 +00:00)


---

# ***Заключение***
### ***По итогам тестов, можно понять, что в большинстве случаев, в том числе при инициализации, лучше всего себя показывает класс написанный при помощи Cython, в некоторых случаях он уступает, хуже всего себя проявил класс и методы написанные на Pure Python, хотя иногда он работает быстрее чем NumPy, таким образом получилось хоть и немного но ускорить процесс вычисления. Но следует помнить и понимать, что каждая задача ситуативна и нужно думать, что будет лучше конкретно для этой задачи.***

---